In [19]:
import requests
import json
from datetime import datetime
import datetime 
import pandas as pd

In [14]:
crawled = list()
papers = list()
affiliations = list()
nav_list = list()
url_params = "&or_facet_publication_type=article&and_facet_year=2023"
url_np = "/discover/publication/results.json?"
url_base = "https://app.dimensions.ai"

In [11]:
def my_parser(data,navigation):
    
    docs = data.get('docs')
    for item in docs:
        title = item.get('title')
        journal_title = item.get('journal_title')
        source_title_id = item.get('source_title_id')
        pub_class = item.get('pub_class')
        publisher = item.get('publisher')
        publisher_place = item.get('publisher_place')
        cited = item.get('cited_dimensions_ids') 
        cited_count = len(cited) if cited is not None else 0
        pub_id = item.get('id')
        pub_date = item.get('pub_date')
        pub_year = item.get('pub_year')
        doi = item.get('doi')
        pmid = item.get('pmid')
        pmcid = item.get('pmc_id')
        pages = item.get('pages')
        issue_count = item.get('issue')
        volume = item.get('volume')
        abstract_url = 'https://app.dimensions.ai'+item['navigation']['abstract_json']
    

        var_dict = {
            'pub_id' : pub_id,
            'title': title,
            'journal_title': journal_title,
            'journal_id': source_title_id,
            'pub_class': pub_class,
            'publisher' : publisher,
            'publisher_place' : publisher_place,
            'cited_count' : cited_count,
            'pub_date' : pub_date,
            'pub_year' : pub_year,
            'doi' : doi,
            'pmid' : pmid,
            'pmcid' : pmcid,
            'pages' : pages,
            'issue_count' : issue_count,
            'volume' : volume,
            'abstract_url' : abstract_url
        }
        papers.append(var_dict)
        nav_list.append({'navigation':navigation,'pub_id':pub_id})

        try:
            for a in item['affiliations_details']:
                fname = a.get('first_name')
                lname = a.get('last_name')
                university = a['affiliations'][0].get('name')
                country = a['affiliations'][0].get('country')
                city = a['affiliations'][0].get('city')


                var_dict = {
                    'pub_id' : pub_id,
                    'first_name': fname,
                    'last_name': lname,
                    'university': university,
                    'country': country,
                    'city' : city
                }
                affiliations.append(var_dict)
        except:
            print(f"An exception occurred ->> navigation:{navigation}")



In [12]:
def pipe(url,request_number):
    try:
        response = requests.request("GET", url, headers=headers, data=payload)
        data = response.json()
        url_np = data['navigation']['results_json']
        url = url_base+url_np+url_params
        print('numer:',request_number,' ->>code: ',response.status_code,' ->>time: ',datetime.now().strftime('%H:%M:%S'),'->>np: ',url_np)
        my_parser(data,url_np)
        return pipe(url,request_number+1)
    except Exception as err:
        print('exception: ',err,' ->>time:',datetime.now().strftime('%H:%M:%S'))
        return url

In [15]:

url = "https://app.dimensions.ai/discover/publication/results.json?&or_facet_year=2023&or_facet_publication_type=article"

payload = {}
headers = {
  'Accept': 'application/json',
  'Accept-Language': 'en-US,en;q=0.9',
  'Cache-Control': 'no-cache, no-store, must-revalidate',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json',
  'Cookie': 'uber_auth_tkt=c38f7baf7a3abe131c16334f8debdfd196994ec7bea3773f0b7edb0b12f9c2f1aaa14189313b5561d0fd438a4d05685773e3f3a8c1c1f376b42ed1b6e0dd34e365c527c3eW9wYWoyMjMxNEBvcHJldm9sdC5jb20%3D!A50519a8823992f4625f21d9e23d194!userid_type:b64unicode; uber_auth_tkt=c38f7baf7a3abe131c16334f8debdfd196994ec7bea3773f0b7edb0b12f9c2f1aaa14189313b5561d0fd438a4d05685773e3f3a8c1c1f376b42ed1b6e0dd34e365c527c3eW9wYWoyMjMxNEBvcHJldm9sdC5jb20%3D!A50519a8823992f4625f21d9e23d194!userid_type:b64unicode; OptanonAlertBoxClosed=2024-02-08T19:13:28.249Z; OptanonConsent=isGpcEnabled=0&datestamp=Thu+Feb+08+2024+22%3A43%3A50+GMT%2B0330+(Iran+Standard+Time)&version=202401.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=67f585d3-6138-4244-a7c4-a48daea5e005&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A0%2CC0003%3A0%2CC0004%3A0&geolocation=NL%3BUT&AwaitingReconsent=false; session=15Wam0V30IJfx-vEJwgkEaYS3LdxQjOZ2A3xAinLPvP8EmLOtqVmjaAWLpn9HPmGDlMx5dlkmBP3qptuJQT8-FsxNzA3NDE5NjYwLCAxNzA3NDE5NTg3LjQzMjgyMTMsIHsiYXV0aGVudGljYXRpb24iOiBudWxsLCAiYXV0aF9tZXRob2QiOiAiZGltZW5zaW9ucyIsICJfY3NyZnRfIjogImVlZWRmOTNhYmIwMjVjYTE4M2I5MmYxMjhjNTUxYmQwMDg5NWRkYzUifV0; session=DDBI4vqitDb7T_RvHJ2NFfF8eAjmDZuEZOhyiBeDcxt31Kq96H1B_IVuC_pCpKcGUTcOsiojVNm2-XJHNrypW1sxNzA3NDE5NzA1LCAxNzA3NDE5NTg3LjQzMjgyMTMsIHsiYXV0aGVudGljYXRpb24iOiBudWxsLCAiYXV0aF9tZXRob2QiOiAiZGltZW5zaW9ucyIsICJfY3NyZnRfIjogImVlZWRmOTNhYmIwMjVjYTE4M2I5MmYxMjhjNTUxYmQwMDg5NWRkYzUifV0',
  'Expires': '0',
  'Operations': 'sticky=GZiv6GZgVY1uEKpKX6GX7gMuNXJZSc8syfuYUZTQ',
  'Pragma': 'no-cache',
  'Referer': 'https://app.dimensions.ai/discover/publication?or_facet_year=2023&or_facet_publication_type=article',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) afari/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
  'X-CSRF-Token': 'eeedf93abb025ca183b92f128c551bd00895ddc5',
  'X-Requested-With': 'XMLHttpRequest',
  'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"'
}

In [26]:
last_url = pipe(url,0)

numer: 0  ->>code:  200  ->>time:  23:20:52 ->>np:  /discover/publication/results.json?np=qfkp5SsgNywwvudD6-vnW9zwjAW4E_0rF2pEx0yPqsjHmje6s4S5KJ3s
An exception occurred ->> navigation:/discover/publication/results.json?np=qfkp5SsgNywwvudD6-vnW9zwjAW4E_0rF2pEx0yPqsjHmje6s4S5KJ3s
An exception occurred ->> navigation:/discover/publication/results.json?np=qfkp5SsgNywwvudD6-vnW9zwjAW4E_0rF2pEx0yPqsjHmje6s4S5KJ3s
numer: 1  ->>code:  200  ->>time:  23:20:54 ->>np:  /discover/publication/results.json?np=nFbISR72L6lNOGOblYihARVYD4YQc4girKpAfqYgSYAZfN0fvXsqZYuk
An exception occurred ->> navigation:/discover/publication/results.json?np=nFbISR72L6lNOGOblYihARVYD4YQc4girKpAfqYgSYAZfN0fvXsqZYuk
numer: 2  ->>code:  200  ->>time:  23:20:56 ->>np:  /discover/publication/results.json?np=XeQsIAvx7nnTis66rwhxPG0vZ_KasqAQ4A9Llgr_8jFRIQJQ78WMOPcg
An exception occurred ->> navigation:/discover/publication/results.json?np=XeQsIAvx7nnTis66rwhxPG0vZ_KasqAQ4A9Llgr_8jFRIQJQ78WMOPcg
An exception occurred ->> navig

In [34]:
df = pd.DataFrame(papers).drop_duplicates()
df['error_url'] = last_url
df.to_csv(f"papers_2023", index=False)

df = pd.DataFrame(affiliations).drop_duplicates()
df['error_url'] = last_url
df.to_csv(f"affiliations_2023", index=False)


In [36]:
len(pd.DataFrame(papers).drop_duplicates())

2020

In [37]:
last_url

'https://app.dimensions.ai/discover/publication/results.json?np=rypUzmpDSQqKwywVozLs2mfm8jEWM3p-C2zkJCBHxGE_jbHxdCs_4k6Mgrs%3D&or_facet_publication_type=article&and_facet_year=2023'

In [38]:
headers = {
  'Accept': 'application/json',
  'Accept-Language': 'en-US,en;q=0.9',
  'Cache-Control': 'no-cache, no-store, must-revalidate',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json',
  'Cookie': 'uber_auth_tkt=c38f7baf7a3abe131c16334f8debdfd196994ec7bea3773f0b7edb0b12f9c2f1aaa14189313b5561d0fd438a4d05685773e3f3a8c1c1f376b42ed1b6e0dd34e365c527c3eW9wYWoyMjMxNEBvcHJldm9sdC5jb20%3D!A50519a8823992f4625f21d9e23d194!userid_type:b64unicode; uber_auth_tkt=c38f7baf7a3abe131c16334f8debdfd196994ec7bea3773f0b7edb0b12f9c2f1aaa14189313b5561d0fd438a4d05685773e3f3a8c1c1f376b42ed1b6e0dd34e365c527c3eW9wYWoyMjMxNEBvcHJldm9sdC5jb20%3D!A50519a8823992f4625f21d9e23d194!userid_type:b64unicode; OptanonAlertBoxClosed=2024-02-08T19:13:28.249Z; OptanonConsent=isGpcEnabled=0&datestamp=Thu+Feb+08+2024+22%3A43%3A50+GMT%2B0330+(Iran+Standard+Time)&version=202401.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=67f585d3-6138-4244-a7c4-a48daea5e005&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A0%2CC0003%3A0%2CC0004%3A0&geolocation=NL%3BUT&AwaitingReconsent=false; session=15Wam0V30IJfx-vEJwgkEaYS3LdxQjOZ2A3xAinLPvP8EmLOtqVmjaAWLpn9HPmGDlMx5dlkmBP3qptuJQT8-FsxNzA3NDE5NjYwLCAxNzA3NDE5NTg3LjQzMjgyMTMsIHsiYXV0aGVudGljYXRpb24iOiBudWxsLCAiYXV0aF9tZXRob2QiOiAiZGltZW5zaW9ucyIsICJfY3NyZnRfIjogImVlZWRmOTNhYmIwMjVjYTE4M2I5MmYxMjhjNTUxYmQwMDg5NWRkYzUifV0; session=DDBI4vqitDb7T_RvHJ2NFfF8eAjmDZuEZOhyiBeDcxt31Kq96H1B_IVuC_pCpKcGUTcOsiojVNm2-XJHNrypW1sxNzA3NDE5NzA1LCAxNzA3NDE5NTg3LjQzMjgyMTMsIHsiYXV0aGVudGljYXRpb24iOiBudWxsLCAiYXV0aF9tZXRob2QiOiAiZGltZW5zaW9ucyIsICJfY3NyZnRfIjogImVlZWRmOTNhYmIwMjVjYTE4M2I5MmYxMjhjNTUxYmQwMDg5NWRkYzUifV0',
  'Expires': '0',
  'Operations': 'sticky=GZiv6GZgVY1uEKpKX6GX7gMuNXJZSc8syfuYUZTQ',
  'Pragma': 'no-cache',
  'Referer': 'https://app.dimensions.ai/discover/publication?or_facet_year=2023&or_facet_publication_type=article',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) brave/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
  'X-CSRF-Token': 'eeedf93abb025ca183b92f128c551bd00895ddc5',
  'X-Requested-With': 'XMLHttpRequest',
  'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"'
}
crawled = list()
papers = list()
affiliations = list()
nav_list = list()

In [39]:
last_url = pipe(last_url,0)

exception:  'navigation'  ->>time: 23:30:00
